In [76]:
!pip install findspark
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType , StructField, StringType, IntegerType

In [77]:
spark = SparkSession.builder.master("local[1]").appName("giveatry").getOrCreate()

df = spark.read.format('csv').options(delimiter=';',header="True",inferschema="True").load("C:\\Users\\brhng\\Desktop\\OnlineRetail.csv")

In [78]:
df.describe().toPandas()

,summary,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,count,541909,541909,541909,541909,541909,541909,541909,541909
1,mean,559965.752026781,27623.240210938104,14.23573883161512,9.55224954743324,None,29.921163668665333,11476.974671024102,None
2,stddev,13428.417280805148,16799.73762842771,543.0146023996443,218.08115785023355,None,595.7455525989111,6777.908326012993,None
3,min,536365,10002,"""ASSORTED FLOWER COLOUR """"LEIS""""""",-80995,1.02.2011 08:23,"-11062,06",0,Australia
4,max,C581569,m,wrongly sold sets,80995,9.12.2011 12:50,"99,96",18287,Unspecified


In [79]:
df.printSchema() 

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [80]:
from pyspark.sql.functions import count
def my_count(df_in):
    df_in.agg(*[count(c).alias(c) for c in df_in.columns]).show()
my_count(df)

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|   541909|   541909|     541909|  541909|     541909|   541909|    541909| 541909|
+---------+---------+-----------+--------+-----------+---------+----------+-------+



In [81]:
df.show()

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     2,55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|1.12.2010 08:26|     2,75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|1.12.2010 08:26|     7,65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|1.12.2010 08:

In [82]:
from pyspark.sql.functions import *
timeFmt = "MM/dd/yy HH:mm"
df = df.withColumn("NewInvoiceData2",  to_timestamp("InvoiceDate","d.MM.yyyy HH:mm"))
df.show()
# df = df.withColumn('NewInvoiceDate', when(col('InvoiceDate').isNotNull(), to_utc_timestamp(unix_timestamp(col('InvoiceDate'), timeFmt).cast('timestamp'), 'UTC')).otherwise(col('InvoiceDate')))

# df.show(5)
     

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+-------------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|    NewInvoiceData2|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+-------------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     2,55|     17850|United Kingdom|2010-12-01 08:26:00|
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|2010-12-01 08:26:00|
|   536365|   84406B|CREAM CUPID HEART...|       8|1.12.2010 08:26|     2,75|     17850|United Kingdom|2010-12-01 08:26:00|
|   536365|   84029G|KNITTED UNION FLA...|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|2010-12-01 08:26:00|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|2010-12-01 08:26:00|
|   5363

In [83]:
from pyspark.sql.functions import round

from pyspark.sql import functions as F

df = df.withColumn("UnitPrice", F.regexp_replace("UnitPrice", ",", ".").cast("double"))

df = df.withColumn('TotalPrice', round(df.Quantity * df.UnitPrice, 2) )

df.show()

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+-------------------+----------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|    NewInvoiceData2|TotalPrice|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+-------------------+----------+
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     2.55|     17850|United Kingdom|2010-12-01 08:26:00|      15.3|
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     3.39|     17850|United Kingdom|2010-12-01 08:26:00|     20.34|
|   536365|   84406B|CREAM CUPID HEART...|       8|1.12.2010 08:26|     2.75|     17850|United Kingdom|2010-12-01 08:26:00|      22.0|
|   536365|   84029G|KNITTED UNION FLA...|       6|1.12.2010 08:26|     3.39|     17850|United Kingdom|2010-12-01 08:26:00|     20.34|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|1.12

In [28]:
from pyspark.sql.functions import to_utc_timestamp,unix_timestamp,lit,datediff,col
from pyspark.sql.functions import date_format

In [96]:
df.show()

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+-------------------+----------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|    NewInvoiceData2|TotalPrice|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+-------------------+----------+
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     2.55|     17850|United Kingdom|2010-12-01 08:26:00|      15.3|
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     3.39|     17850|United Kingdom|2010-12-01 08:26:00|     20.34|
|   536365|   84406B|CREAM CUPID HEART...|       8|1.12.2010 08:26|     2.75|     17850|United Kingdom|2010-12-01 08:26:00|      22.0|
|   536365|   84029G|KNITTED UNION FLA...|       6|1.12.2010 08:26|     3.39|     17850|United Kingdom|2010-12-01 08:26:00|     20.34|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|1.12

In [101]:
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

from pyspark.sql.functions import mean, min, max, sum, datediff, to_date

date_max = df.select(max('NewInvoiceDate2')).toPandas()

current = to_utc_timestamp(unix_timestamp(lit(str(date_max.iloc[0][0])), 'yy-MM-dd HH:mm').cast('timestamp'), 'UTC')

df = df.withColumn('Duration', datediff(lit(current), 'NewInvoiceDate'))

#Recency, Frequency, Monetary

recency = df.groupBy('CustomerID').agg(min('Duration').alias('Recency'))

frequency = df.groupBy('CustomerID', 'InvoiceNo').count()\
    .groupBy('CustomerID')\
    .agg(count('*').alias("Frequency"))

monetary = df.groupBy('CustomerID').agg(round(sum('TotalPrice'), 2).alias('Monetary'))

rfm = recency.join(frequency, 'CustomerID', how = 'inner')\
    .join(monetary, 'CustomerID', how = 'inner')

rfm.show()

AnalysisException: cannot resolve '`NewInvoiceDate2`' given input columns: [Country, CustomerID, Description, InvoiceDate, InvoiceNo, NewInvoiceData2, Quantity, StockCode, TotalPrice, UnitPrice];;
'Aggregate [max('NewInvoiceDate2) AS max(NewInvoiceDate2)#4497]
+- Project [InvoiceNo#3382, StockCode#3383, Description#3384, Quantity#3385, InvoiceDate#3386, UnitPrice#4122, CustomerID#3388, Country#3389, NewInvoiceData2#4050, round((cast(Quantity#3385 as double) * UnitPrice#4122), 2) AS TotalPrice#4132]
   +- Project [InvoiceNo#3382, StockCode#3383, Description#3384, Quantity#3385, InvoiceDate#3386, cast(regexp_replace(UnitPrice#3387, ,, .) as double) AS UnitPrice#4122, CustomerID#3388, Country#3389, NewInvoiceData2#4050]
      +- Project [InvoiceNo#3382, StockCode#3383, Description#3384, Quantity#3385, InvoiceDate#3386, UnitPrice#3387, CustomerID#3388, Country#3389, to_timestamp('InvoiceDate, Some(d.MM.yyyy HH:mm)) AS NewInvoiceData2#4050]
         +- RelationV2[InvoiceNo#3382, StockCode#3

In [75]:
df = df.withColumn('TotalPrice', round(df.Quantity.cast(DoubleType()) * df.LOW), 2)

AttributeError: 'DataFrame' object has no attribute 'LOW'

In [52]:
df = df.withColumn('UnitPrice2',df.UnitPrice.cast("double"))

In [65]:
from pyspark.sql.types import DoubleType
df.withColumn("UnitPrice2",col("UnitPrice").cast(DoubleType())).show()

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+----------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|UnitPrice2|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+----------+
|   536365|   85123A|WHITE HANGING HEA...|       6|1.12.2010 08:26|     2,55|     17850|United Kingdom|      null|
|   536365|    71053| WHITE METAL LANTERN|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|      null|
|   536365|   84406B|CREAM CUPID HEART...|       8|1.12.2010 08:26|     2,75|     17850|United Kingdom|      null|
|   536365|   84029G|KNITTED UNION FLA...|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|      null|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|1.12.2010 08:26|     3,39|     17850|United Kingdom|      null|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|1.12.2010 08:26|     7,65|   

In [68]:
from pyspark.sql import functions as F

df3 = df.withColumn("LOW", F.regexp_replace("UnitPrice", ",", ".").cast("double"))

In [97]:
df.dtypes

[('InvoiceNo', 'string'),
 ('StockCode', 'string'),
 ('Description', 'string'),
 ('Quantity', 'int'),
 ('InvoiceDate', 'string'),
 ('UnitPrice', 'double'),
 ('CustomerID', 'int'),
 ('Country', 'string'),
 ('NewInvoiceData2', 'timestamp'),
 ('TotalPrice', 'double')]

In [87]:
def describe_pd(df_in, columns, style):
    '''
    Function to union the basic stats results and deciles
    :param df_in: the input dataframe 
    :param columns: the cloumn name list of the numerical variable     
    :param style: the display style  

    :return : the numerical describe info. of the input dataframe   

    :author: Wenqiang Feng
    :email:  von198@gmail.com  
    '''       

    if style == 1:
        percentiles = [25, 50, 75]
    else:
        percentiles = np.array(range(0, 110, 10))
    
    percs = np.transpose([np.percentile(df_in.select(x).collect(), percentiles) for x in columns])
    percs = pd.DataFrame(percs, columns=columns)
    percs['summary'] = [str(p) + '%' for p in percentiles]
    
    spark_describe = df_in.describe().toPandas()
    new_df = pd.concat([spark_describe, percs],ignore_index=True)
    new_df = new_df.round(2)
    return new_df[['summary'] + columns]

In [88]:
rdd.show()

NameError: name 'rdd' is not defined